# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## Загрузка данных

In [2]:
import pandas as pd
import numpy as np
#from sklearn.datasets import make_spd_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score
from numpy.linalg import inv, cond, det
from numpy.random import RandomState


import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('/datasets/insurance.csv')

In [4]:
display(data.info())
display(data.head(5))
display('Количество явных дубликтов:', data.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


'Количество явных дубликтов:'

153

Предоставленные данные состоят из 5 столбцов. 4 из них - признаки, а один - Страховые выплаты - целевой признак. Пропущенных значений нет.
Дубликаты есть. Так как совпадение по всем признакам крайне маловероятно, избавимся от них.

In [5]:
data['Возраст'] = data['Возраст'].astype('int')
data['Зарплата'] = data['Зарплата'].astype('int')
data.dtypes

Пол                  int64
Возраст              int64
Зарплата             int64
Члены семьи          int64
Страховые выплаты    int64
dtype: object

Столбцы 'Возраст' и 'Зарплата'  думаю лучше перевести в целочисленный вид.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

Докажем, что:

$$
a = Xw = X'w' = XPw' = a'
$$

Чтобы это равенство выполнялось, необходимо соблюсти условие:

$$
w' = P^{-1} w
$$


Получим следующее равенство:

$$
a = Xw = X'(P^{-1} w) = XP(P^{-1} w) = XEw = Xw = a
$$

**Ответ:** Нет, не изменится.

**Обоснование:** Обратная для квадратной матрицы P — матрица P с верхним индексом -1, произведение которой на P равно единичной матрице. Если любую матрицу умножить на единичную (или наоборот), получится эта же матрица. Поэтому качество линейной регрессии не изменится - матрицы будут одинаковы.


Формула расчета весов для линейной регрессии:

$$
w = (X^T X)^{-1} X^T y
$$

Представим новую матрицу признаков $X1$ как произведение старой $X$ на матрицу $P$ (обратимую):

$$
X1 = X * P
$$
Подставим новое значение X1 в формулу $w1$:

$$
w1 = ((X P)^T X P)^{-1} (X P)^T y
$$
Раскроем первое произведение $ (XP)^T $:

$$
w1 = (P^T X^T X P)^{-1} P^T X^T y
$$
Перегруппируем множители в скобках, чтобы это выглядело как произведение трех множителей:

$$
w1 = (P^T (X^T X) P)^{-1} P^T X^T y
$$
Раскроем скобки $ (P^T (X^T X) P)^{-1} $:

$$
w1 = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
Так как $P$ по условию обратимая, то произведение $(P^T)^{-1} P^T$ равно $E$ (единичная матрица):

$$
w1 = P^{-1} (X^T X)^{-1} E X^T y = P^{-1} (X^T X)^{-1} X^T y
$$
Можно заметить, что справа получилась формула для $w$:

$$
w1 = P^{-1} (X^T X)^{-1} X^T y = P^{-1} w
$$

## Алгоритм преобразования

**Алгоритм**

Создадим случайную обратимую матрицу P, затем умножим матрицу с признаками X на матрицу P.

$$
X@P=Z
$$

Для возврата к исходной матрице X, умножим новую матрицу Z на обратную матрицу P^-1.

$$
Z@P^{-1}=X
$$

**Обоснование**
Как было доказано в шаге 2, качество линейной регрессии не изменится при умножении матрицы признаков на любую обратимую матрицу P. Поэтому можно следовать описанному выше алгоритму без опасения получить другой результат линейной регрессии.

## Проверка алгоритма

Выделим из датасета набор признаков и целевой признак.

In [6]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

Получим матрицу из значений набора признаков.

In [7]:
features_matrix = features.values
features_matrix

array([[    1,    41, 49600,     1],
       [    0,    46, 38000,     1],
       [    0,    29, 21000,     0],
       ...,
       [    0,    20, 33900,     2],
       [    1,    22, 32700,     3],
       [    1,    28, 40600,     1]])

Создадим рандомную матрицу нужного размера

In [8]:
random_matrix = np.random.normal(3,size=(4,4))
display(random_matrix)

array([[4.46201551, 1.76551356, 4.517603  , 2.19405044],
       [2.48589271, 2.18714045, 3.10722211, 1.82427286],
       [3.54653478, 2.15987341, 2.30255531, 2.96077874],
       [3.68381173, 4.49756573, 2.41729188, 3.73917543]])

Проверим матрицу на обратимость - вероятность сгенерировать необратимую матрицу очень мала, но все же. При использовании необратимой матрицы  алгоритм работать не будет. Для проверки будем использовать функцию __numpy.linalg.inv()__, если матрица необратима, то код упадет с ошибкой.

In [9]:
inv(random_matrix)

array([[ 1.12210087, -1.66462973, -0.69095726,  0.70084021],
       [ 0.39340648, -0.43968069, -0.95654379,  0.7410881 ],
       [-0.39935528,  1.11168079,  0.24326722, -0.50066222],
       [-1.3205108 ,  1.45016421,  1.6740127 , -0.99075563]])

In [10]:
det(random_matrix)

8.993817378643326

Матрица обратима. Определитель матрицы не равен 0 

Перемножим матрицы.

In [11]:
transformed_matrix = features_matrix @ random_matrix
features_transformed = list(transformed_matrix)

Разделим набор признаков и целевой признак исходного датасета на обучающую и тестовую выборки.

In [12]:
features_train_origin, features_test_origin, target_train_origin, target_test_origin = train_test_split(features,target, test_size=0.25, random_state = 42)

<div class="alert alert-info"> <b>Комментарий студента:</b> Внес небольшие исправления - поделил на тренировочную и тестовую оригинальный выборки одной строкой.
    По поводу random_state - почему-то он не применялся когда указывался через константу. </div>


Создадим модели.

In [13]:
lro = LinearRegression()
lrt = LinearRegression()

Обучим первую модель, сделаем предсказание и оценим качество модели.

In [14]:
lro.fit(features_train_origin, target_train_origin)
predicted_origin = lro.predict(features_test_origin)
print('R2 исходного набора данных:', r2_score(target_test_origin,predicted_origin))

R2 исходного набора данных: 0.4254778535754763


Разделим набор признаков преобразованного датасета на обучающую и тестовую выборки. Обучим вторую модель, сделаем предсказание и оценим качество модели.

In [15]:
features_train_transformed, features_test_transformed = train_test_split(features_transformed, test_size=0.25, random_state =42)

In [16]:
lrt.fit(features_train_transformed, target_train_origin)
predicted_transformed = lrt.predict(features_test_transformed)
print("R2 преобразованного набора данных:", r2_score(target_test_origin,predicted_transformed))

R2 преобразованного набора данных: 0.4254778535765814


Показатели R2 низкие, но отличаются незначительно.

Попробуем восстановить исходную матрицу, используя формулу из шага 3.

$$
Z@P^{-1}=X
$$

In [17]:
final_matrix = transformed_matrix.dot(inv(random_matrix))

In [18]:
#сравним округленные матрицы
np.round(final_matrix) == np.round(features_matrix)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       ...,
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

Матрицы одинаковы. Мы реализовали алгоритм, описанный в шаге 3 и доказали, что качество линейной регрессии не меняется при умножении матрицы признаков на любую обратимую матрицу P.